In [0]:
!git clone https://github.com/tensorflow/models.git
!git clone https://github.com/liloi/bert-tf2.git
!pip install tf-nightly-gpu
!pip install --user -r /content/models/official/requirements.txt
# Click "RESTART RUNTIME" button to restart your colab runtime environment.

In [0]:
# replace data process file to yourself file
!mv ./bert-tf2/modified_code/classifier_data_lib.py models/official/nlp/bert/classifier_data_lib.py
!mv ./bert-tf2/modified_code/create_finetuning_data.py models/official/nlp/bert/create_finetuning_data.py

In [0]:
%set_env PYTHONPATH=/content/models
%set_env TASK_NAME=BDBK
%set_env DATA_DIR=/content/bert-tf2/train_data
%set_env OUTPUT_DIR=/content/output
%set_env CONFIG_FILE=/content/bert-tf2/config_file

In [0]:
!python ./models/official/nlp/bert/create_finetuning_data.py \
    --input_data_dir=${DATA_DIR} \
    --vocab_file=${CONFIG_FILE}/vocab.txt \
    --train_data_output_path=${DATA_DIR}/${TASK_NAME}_train.tf_record \
    --eval_data_output_path=${DATA_DIR}/${TASK_NAME}_eval.tf_record \
    --meta_data_file_path=${DATA_DIR}/${TASK_NAME}_meta_data \
    --fine_tuning_task_type=classification \
    --max_seq_length=128 \
    --classification_task_name=${TASK_NAME}

In [0]:
!python ./models/official/nlp/bert/run_classifier.py \
    --mode='train_and_eval' \
    --input_meta_data_path=${DATA_DIR}/${TASK_NAME}_meta_data \
    --train_data_path=${DATA_DIR}/${TASK_NAME}_train.tf_record \
    --eval_data_path=${DATA_DIR}/${TASK_NAME}_eval.tf_record \
    --bert_config_file=${CONFIG_FILE}/bert_config.json \
    --train_batch_size=4 \
    --eval_batch_size=4 \
    --steps_per_loop=1 \
    --learning_rate=2e-5 \
    --num_train_epochs=3 \
    --model_dir=${OUTPUT_DIR} \
    --distribution_strategy=mirrored \
    --hub_module_url=https://tfhub.dev/tensorflow/bert_zh_L-12_H-768_A-12/1 \
    --use_keras_compile_fit=True \
    --num_gpus=1